In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.porter import PorterStemmer

In [2]:
def indiscernibility(matrix,y):
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    y=list(df.columns)
    grouped_df=df.groupby(y)
    
    ind_R=list(list())
    for key, item in grouped_df:
#         print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

def encoding_discourse_type(x):
    if x=="Lead":
        return 0
    if x=="Position":
        return 1
    if x=="Evidence":
        return 2
    if x=="Claim":
        return 3
    if x=="Concluding Statement":
        return 4
    if x=="Counterclaim":
        return 5
    if x=='Rebuttal':
        return 6
    
def stemming_stopwords_removing(df):
    corpus=[]
    for i in range(len(df)):
        review=re.sub('[^a-zA-Z]',' ',df["discourse_text"][i])
        review=review.lower()
        review=review.split()
        ps=PorterStemmer()
        all_stopwords=stopwords.words("english")
        review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
        review=' '.join(review)
        corpus.append(review)
    return corpus

# storing the total occurrence.......
def get_total_index_words(corpus):
    index_word={}
    for i in corpus:
        s=i.split()
        for j in s:
            if j not in index_word:
                index_word[j]=1
            else:
                index_word[j]+=1
    return index_word
    
def get_values(dataset,threshold=1):
    
    # taking sample of 20 documents for lead category....
    df = dataset

    total_corpus = stemming_stopwords_removing(df)
    # print(total_corpus)

    # getting total index words and their count in the taken sample as a dict
    total_index_words = get_total_index_words(total_corpus)
    # print(len(lead_index_words))

    # Creating a list of total keywords before filtering..
    total_keywords = list(total_index_words.keys())

    # Creating a matrix of width equals len(lead_keywords)
    matrix=np.zeros((len(df),len(total_keywords)))

    
    # Storing occurrence of each term in each document respectively
    for i in range(len(total_corpus)):
        s = total_corpus[i].split()
        for h in s:
            j = total_keywords.index(h)
            matrix[i,j] += 1


    # Storing their weights....
    weighted_matrix = np.copy(matrix)
    
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            weighted_matrix[i,j] = weighted_matrix[i,j] / total_index_words[total_keywords[j]]
    #            print(weighted_matrix[i,j])



    # FILTERING WEIGHTS with a threshold.......
    valid_index = []
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            if weighted_matrix[i,j] >= threshold:
                valid_index.append(j)

    # removing duplicates and storing them in a list.......    
    valid_index = list(set(valid_index))


    # # Storing the final keywords.... 
    valid_index_words = []
    for i in range(len(valid_index)):
        valid_index_words.append(total_keywords[valid_index[i]])
    # print(valid_lead_index_words)

    return total_keywords,total_index_words,matrix, weighted_matrix, valid_index, valid_index_words

def get_test_matrix(dataset,total_sample_keywords,threshold=1):
      # taking sample of 20 documents for lead category....
    df = dataset

    total_corpus = stemming_stopwords_removing(df)
    # print(total_corpus)

    # getting total index words and their count in the taken sample as a dict
    total_index_words = get_total_index_words(total_corpus)


    # Creating a list of total keywords before filtering..
    total_keywords = total_sample_keywords
    # print(lead_keywords)

    # Creating a matrix of width equals len(lead_keywords)
    matrix=np.zeros((len(df),len(total_keywords)))
    print(len(df),len(total_keywords))

    
    # Storing occurrence of each term in each document respectively
    for i in range(len(total_corpus)):
        s = total_corpus[i].split()
        for h in s:
            try:
                j = total_keywords.index(h)
                matrix[i,j] += 1
            except:
                continue
            
    # Storing their weights....
    weighted_matrix = np.copy(matrix)
    
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            try:
                weighted_matrix[i,j] = weighted_matrix[i,j] / total_index_words[total_keywords[j]]
            except:
                continue


    # FILTERING WEIGHTS with a threshold.......
    valid_index = []
    for i in range(len(df)):
        for j in range(len(total_keywords)):
            if weighted_matrix[i,j] >= threshold:
                valid_index.append(j)

    # removing duplicates and storing them in a list.......    
    valid_index = list(set(valid_index))
    


    # # Storing the final keywords.... 
    valid_index_words = []
    for i in range(len(valid_index)):
        valid_index_words.append(total_keywords[valid_index[i]])
    # print(valid_lead_index_words)

    return total_keywords,total_index_words,matrix, weighted_matrix, valid_index, valid_index_words


In [3]:
def single_element_indiscernibility(matrix,index):
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    y=list(df.columns)
    
    grouped_df=df.groupby(y[index])
    
    ind_R=list(list())
    for key, item in grouped_df:
    #print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

def indiscernibility(matrix,y):
    if len(y)==0:
        return []
    df=pd.DataFrame(matrix)
    # this will return the list of columns     
    grouped_df=df.groupby(y)
    ind_R=list(list())
    for key, item in grouped_df:
    #print(grouped_df.get_group(key), "\n",grouped_df.get_group(key).index ,"\n\n")
        lis=[]
        for i in grouped_df.get_group(key).index:
            lis.append(i)
        ind_R.append(list(lis))
    return ind_R

    

In [4]:
train=pd.read_csv("train.csv")
train_head=train.head(5000)
train_tail=train.tail(5000)
train=pd.concat([train_head,train_tail])
train.reset_index(inplace=True)
train

,index,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...,...
9995,144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...
9996,144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...
9997,144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838
9998,144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...


In [5]:
c_attr=np.array([[1,0,2,2,1],[0,1,1,1,0],[2,0,0,1,2],[1,1,0,2,1],[1,0,2,0,1],[2,2,0,1,2],[2,1,1,1,2],[0,1,1,0,0]])
dec_attr=np.array([0,2,1,2,1,1,2,1])
terms=np.array(["a","b","c","d"])


In [6]:
# NOW REMOVING IDENTICAL COLUMNS FROM OUR MATRIX INPLACE.......
print((c_attr))
print("shape of old training attribute",c_attr.shape)
grp=np.column_stack(c_attr)
temp=pd.DataFrame(grp)
# print(temp.shape)

temp_grp=indiscernibility(temp,list(temp.columns))
# print(temp_grp)

unique_columns=[]
for i in range(len(temp_grp)):
    unique_columns.append(temp_grp[i][0])
    
unique_columns.sort()
print()
print("Unique columns:",unique_columns)
print("\nNo of unique columns:",len(unique_columns))

unique_col_table=list(list())
for i in unique_columns:
    unique_col_table.append(list(grp[i]))
unique_col_table=np.array(unique_col_table)
# print("\nunique_col_table:")
print(unique_col_table)
print("\n unique_col_table shape",unique_col_table.shape)

new_train_attributes=np.column_stack(unique_col_table)

print("\nnew train attributes after preserving only one repetitve element:\n",new_train_attributes)
print("\nshape of new train attributes:",new_train_attributes.shape)
c_attr=new_train_attributes.copy()

repetitive_columns=list(set(list(range(len(c_attr[0]))))-set(unique_columns))
print("number of repeated columns are: ", len(repetitive_columns))

[[1 0 2 2 1]
 [0 1 1 1 0]
 [2 0 0 1 2]
 [1 1 0 2 1]
 [1 0 2 0 1]
 [2 2 0 1 2]
 [2 1 1 1 2]
 [0 1 1 0 0]]
shape of old training attribute (8, 5)

Unique columns: [0, 1, 2, 3]

No of unique columns: 4
[[1 0 2 1 1 2 2 0]
 [0 1 0 1 0 2 1 1]
 [2 1 0 0 2 0 1 1]
 [2 1 1 2 0 1 1 0]]

 unique_col_table shape (4, 8)

new train attributes after preserving only one repetitve element:
 [[1 0 2 2]
 [0 1 1 1]
 [2 0 0 1]
 [1 1 0 2]
 [1 0 2 0]
 [2 2 0 1]
 [2 1 1 1]
 [0 1 1 0]]

shape of new train attributes: (8, 4)
number of repeated columns are:  0


In [7]:
combined_attr=c_attr.copy()
new_arr=np.array([])
for i in range(len(combined_attr)):
    new_arr=np.append(new_arr,np.append(combined_attr[i],dec_attr[i]))
# now reshape it...
new_arr=new_arr.reshape(len(dec_attr),len(c_attr[0])+1)
combined_attr=new_arr.copy()
combined_attr
# df1=pd.DataFrame(combined_attr)
# print(list(df1.columns))

array([[1., 0., 2., 2., 0.],
       [0., 1., 1., 1., 2.],
       [2., 0., 0., 1., 1.],
       [1., 1., 0., 2., 2.],
       [1., 0., 2., 0., 1.],
       [2., 2., 0., 1., 1.],
       [2., 1., 1., 1., 2.],
       [0., 1., 1., 0., 1.]])

In [8]:
for i in range(len(terms)):
    print(terms[i],":",single_element_indiscernibility(c_attr,i))
    

a : [[1, 7], [0, 3, 4], [2, 5, 6]]
b : [[0, 2, 4], [1, 3, 6, 7], [5]]
c : [[2, 3, 5], [1, 6, 7], [0, 4]]
d : [[4, 7], [1, 2, 5, 6], [0, 3]]


In [9]:
# THIS CELL IS ONLY FOR EXPERIMENT PURPOSE...
arr=[0,1,2,3]
ind=indiscernibility(c_attr,arr)
print("\nIndiscernibility:",ind)
res=[arr[i:j] for i in range(len(arr))
    for j in range(i+1,len(arr)+1)]

for i in res:
    print(i,":",indiscernibility(c_attr,i))


Indiscernibility: [[7], [1], [4], [0], [3], [2], [6], [5]]
[0] : [[1, 7], [0, 3, 4], [2, 5, 6]]
[0, 1] : [[1, 7], [0, 4], [3], [2], [6], [5]]
[0, 1, 2] : [[1, 7], [0, 4], [3], [2], [6], [5]]
[0, 1, 2, 3] : [[7], [1], [4], [0], [3], [2], [6], [5]]
[1] : [[0, 2, 4], [1, 3, 6, 7], [5]]
[1, 2] : [[2], [0, 4], [3], [1, 6, 7], [5]]
[1, 2, 3] : [[2], [4], [0], [3], [7], [1, 6], [5]]
[2] : [[2, 3, 5], [1, 6, 7], [0, 4]]
[2, 3] : [[2, 5], [3], [7], [1, 6], [4], [0]]
[3] : [[4, 7], [1, 2, 5, 6], [0, 3]]


In [10]:
# find R_e as given in the doc......
R_e=single_element_indiscernibility(combined_attr,len(combined_attr[0])-1)
R_e
# Dependency if R_e is always equal to 1.

[[0], [2, 4, 5, 7], [1, 3, 6]]

In [11]:
def calculate_LA(R_e,ind):
    nR_e=list(set())
    nind=list(set())
    ans=[]
    for i in R_e:
        nR_e.append(set(i))
    for i in ind:
        nind.append(set(i))
    
    for i in range(len(nind)):
        for j in range(len(nR_e)):
            if nind[i]&nR_e[j]==nind[i]:
                for val in nind[i]:
                    ans.append(val)
    ans=list(set(ans))
    return ans

def calculate_dependency(R_e,ind,universe=list(range(len(dec_attr)))):
     # here universe is set of all objects or terms.
    if len(ind)==0:
        return 0
    ans=calculate_LA(R_e,ind)
    return len(ans)/len( universe)
                

In [12]:
# THIS CELL IS ONLY FOR EXPERIMENT PURPOSE...CALCULATING DEPENDENCY ----->
arr=[0,1,2,3]
res=[arr[i:j] for i in range(len(arr))
    for j in range(i+1,len(arr)+1)]
print(res)
print(c_attr)
# calculating dependency for all subarrays in array.......
print("calculating dependency for all subarrays in array.......")
print()
for i in res:
    ind=indiscernibility(c_attr,i)
    print(calculate_dependency(R_e,ind))
    
# calculating dependency for all single elements.......
print()
print("calculating dependency for all single elements.......")
print()

for i in range(len(terms)):
    ind=indiscernibility(c_attr,[i])
    print(calculate_dependency(R_e,ind))
    

[[0], [0, 1], [0, 1, 2], [0, 1, 2, 3], [1], [1, 2], [1, 2, 3], [2], [2, 3], [3]]
[[1 0 2 2]
 [0 1 1 1]
 [2 0 0 1]
 [1 1 0 2]
 [1 0 2 0]
 [2 2 0 1]
 [2 1 1 1]
 [0 1 1 0]]
calculating dependency for all subarrays in array.......

0.0
0.5
0.5
1.0
0.125
0.375
1.0
0.0
1.0
0.25

calculating dependency for all single elements.......

0.0
0.125
0.0
0.25


In [13]:
c_attr

array([[1, 0, 2, 2],
       [0, 1, 1, 1],
       [2, 0, 0, 1],
       [1, 1, 0, 2],
       [1, 0, 2, 0],
       [2, 2, 0, 1],
       [2, 1, 1, 1],
       [0, 1, 1, 0]])

In [14]:
# QUICK REDUCT ALGO
C=[]
for i in range(len(c_attr[0])):
    C.append(i)
C=set(C)
R=set()
T=set()

dep_R=0
dep_T=0
dep_Re=calculate_dependency(R_e,indiscernibility(c_attr,list(C)))
while not dep_R==1:
    T=R.copy()
    for x in C-R:
        dep_R=calculate_dependency(R_e,indiscernibility(c_attr,list(R.union({x}))))
        if dep_R>dep_T:
            T=R.union({x}).copy()
            dep_T=calculate_dependency(R_e,indiscernibility(c_attr,list(T)))
    R=T.copy()
    dep_R=calculate_dependency(R_e,indiscernibility(c_attr,list(R)))
    
print(R)
print(calculate_dependency(R_e,indiscernibility(c_attr,list(R))))

# computing new reduced matrix
print("Old conditional matrix before reducing:\n",c_attr)
new_matrix=c_attr[:,list(R)]
print("\nNew conditional matrix after reducing:\n",new_matrix)
# Note: decision attribute will remain the same
print("\nDecision matrix is:\n",dec_attr)

# computing new combined matrix
c_attr=new_matrix.copy()
combined_attr=c_attr.copy()
new_arr=np.array([])
for i in range(len(combined_attr)):
    new_arr=np.append(new_arr,np.append(combined_attr[i],dec_attr[i]))
# now reshape it...
new_arr=new_arr.reshape(len(dec_attr),len(c_attr[0])+1)
combined_attr=new_arr.copy()
print("\nNew combined matrix:\n",combined_attr)

# Storing final reduced columns list
reduced_col=list(R)

# storing new_terms
new_terms=[]
for i in reduced_col:
    new_terms.append(terms[i])
print("\nNew columns as per order:\n",new_terms)
new_train_attributes=c_attr.copy()

{1, 3}
1.0
Old conditional matrix before reducing:
 [[1 0 2 2]
 [0 1 1 1]
 [2 0 0 1]
 [1 1 0 2]
 [1 0 2 0]
 [2 2 0 1]
 [2 1 1 1]
 [0 1 1 0]]

New conditional matrix after reducing:
 [[0 2]
 [1 1]
 [0 1]
 [1 2]
 [0 0]
 [2 1]
 [1 1]
 [1 0]]

Decision matrix is:
 [0 2 1 2 1 1 2 1]

New combined matrix:
 [[0. 2. 0.]
 [1. 1. 2.]
 [0. 1. 1.]
 [1. 2. 2.]
 [0. 0. 1.]
 [2. 1. 1.]
 [1. 1. 2.]
 [1. 0. 1.]]

New columns as per order:
 ['b', 'd']


In [15]:
# Calculation fo lower approximation starts here...
groups_index={}
new_ind=indiscernibility(c_attr,list(range(len(c_attr[0]))))
ind=new_ind.copy()
print(new_ind)
grp=0 # iam following grp index from 0...
for i in new_ind:
  for j in range(len(i)):
    groups_index[i[j]]=grp
  grp+=1
print(groups_index)


[[4], [2], [0], [7], [1, 6], [3], [5]]
{4: 0, 2: 1, 0: 2, 7: 3, 1: 4, 6: 4, 3: 5, 5: 6}


In [16]:

# Step -1: Find all the rows under one category....
def get_class(grp,y):
  lead_index=[False]*len(y)
  for i in range(len(y)):
    if y[i]==grp:
      lead_index[i]=True

  return lead_index

def get_lower_approximation(grp,ind,y):
  lead_index=get_class(grp,y)
  lower_approximation=[]
  for i in ind:
    temp=True
    for j in range(len(i)):
      if not lead_index[i[j]]:
        temp=False
        break
    if temp:
      for j in range(len(i)):
        lower_approximation.append(i[j])
  return lower_approximation

def get_upper_approximation(grp,ind,y):
  lead_index=get_class(grp,y)
  upper_approximation=[]
  for i in ind:
    temp=False
    for j in range(len(i)):
      if lead_index[i[j]]:
        temp=True
        break
    if temp:
      for j in range(len(i)):
        upper_approximation.append(i[j])
  return upper_approximation
  

In [17]:
# Finding lower apporximation for all the different classes we have and storing it in a matrix....
decision_attr=dec_attr.copy()
y=decision_attr.copy()

def get_lower_approximation_matrix(ind,decision_attr):
  lower_approx_matrix=list(list())
  y1=np.unique(y)
  for i in y1:
    lower_approx_matrix.append(list(get_lower_approximation(i,ind,y)))
  return lower_approx_matrix

lower_approx_matrix=get_lower_approximation_matrix(ind,y)

print("lower approximation matrix is:",lower_approx_matrix)

    

# Finding upper apporximation for all the different classes we have and storing it in a matrix....
def get_upper_approximation_matrix(ind,y):
  upper_approx_matrix=list(list())
  y1=np.unique(y)
  for i in y1:
    upper_approx_matrix.append(list(get_upper_approximation(i,ind,y)))
  return upper_approx_matrix

upper_approx_matrix=get_upper_approximation_matrix(ind,y)

print("upper approximation matrix is:",upper_approx_matrix)

# Defining uuniverse again as U:
U=list(range(0,len(y)))
universe=set(U)


# Finding boundary region for each classes....  "MAYBE" region
# Now from here on we will use set() datastructure in python and its predefined operations for further calculation....

def get_boundary_region_matrix(upper_approx_matrix,lower_approx_matrix):
  boundary_reg_matrix=list(list())
  for i in range(len(upper_approx_matrix)):
    upper_set=set(upper_approx_matrix[i])
    lower_set=set(lower_approx_matrix[i])
    boundary_reg_matrix.append(list(upper_set-lower_set))
  return boundary_reg_matrix

boundary_reg_matrix=get_boundary_region_matrix(upper_approx_matrix,lower_approx_matrix)

print("boundary region matrix is:",boundary_reg_matrix)
# now note boundary region for any class is zero which means that they have equal lower and upper approximation


# Finding outside region for each class.... "NO" region
def get_outside_region_matrix(universe,upper_approx_matrix):
  outside_reg_matrix=list(list())
  for i in range(len(upper_approx_matrix)):
    upper_set=set(upper_approx_matrix[i])
    outside_reg_matrix.append(list(universe-upper_set))
  return outside_reg_matrix

outside_reg_matrix=get_outside_region_matrix(universe,upper_approx_matrix)
print("outside region matrix is:",outside_reg_matrix)


lower approximation matrix is: [[0], [4, 2, 7, 5], [1, 6, 3]]
upper approximation matrix is: [[0], [4, 2, 7, 5], [1, 6, 3]]
boundary region matrix is: [[], [], []]
outside region matrix is: [[1, 2, 3, 4, 5, 6, 7], [0, 1, 3, 6], [0, 2, 4, 5, 7]]


In [18]:
def get_set(approx_matrix,attributes):
    res=list(set())
    temp=[]
    for i in approx_matrix:
        for j in i:
            temp=np.where(attributes[j]!=0)
            temp=np.unique(temp)
        res.append(set(temp))
    return res

attr_lower_approx=get_set(lower_approx_matrix,new_train_attributes)
attr_upper_approx=get_set(upper_approx_matrix,new_train_attributes)
attr_boundary_approx=get_set(boundary_reg_matrix,new_train_attributes)
attr_outside_approx=get_set(outside_reg_matrix,new_train_attributes)
print(attr_lower_approx)
print(attr_upper_approx)
print(attr_boundary_approx)
print(attr_outside_approx)

[{1}, {0, 1}, {0, 1}]
[{1}, {0, 1}, {0, 1}]
[set(), set(), set()]
[{0}, {0, 1}, {0}]


In [19]:
# REMOVING INCONSISTENCY TUPLES FROM OUR MATRICES using accuracy method...
# STEP-1: Find the inconsistent rows.....
# NOTE: Deletion of rows is done inplace..so runnnig this cell more than once will give undesirable result

# !--NotebookApp.iopub_data_rate_limit=1.0e10

def get_inconsistent_rows(ind,y):
  inconsistent_rows=list(list())

  for i in ind:
    if len(i)>1:
      temp=y[i[0]]
      for j in i:
        if y[j]!=temp:
          inconsistent_rows.append(list(i))
  return inconsistent_rows
      

inconsistent_rows=get_inconsistent_rows(ind,y)
# print(inconsistent_rows)

def find_accuracy(inconsistent_rows,lower_approx_matrix):
  highest_accuracy=-1
  accuracies=list(list())
  if len(inconsistent_rows)>0:
    # print(inconsistent_rows)
    
    for i in inconsistent_rows:
      lis=[]
      for j in i:
        cat=y[j]
        lower=lower_approx_matrix[cat]
        accuracy=len(lower)/len(U)
        lis.append(accuracy)
      accuracies.append(list(lis))
  return accuracies

accuracies=find_accuracy(inconsistent_rows,attr_lower_approx)

# now note this.. len(inconsistent_rows) and redundant rows will be same...
# so each element  in inconsistent_rows has its accuracy stored in accuracies...in the same format that of inconsistent_rows

  
def find_redundant_rows(accuracies,inconsistent_rows):
  redundant_rows=[]
  for i in range(len(accuracies)):
    max_accuracy=max(accuracies[i])
    for j in range(len(accuracies[i])):
      if accuracies[i][j]!=max_accuracy:
        redundant_rows.append(inconsistent_rows[i][j])
  return redundant_rows
redundant_rows=find_redundant_rows(accuracies,inconsistent_rows)
# print(len(redundant_rows))
# print(redundant_rows)


# Removing redundant rows if any...

if len(redundant_rows)>0:
  new_train_attributes=np.delete(new_train_attributes,redundant_rows,axis=0)

print("Done......")
# DELETEING SAME ROWS FROM decision_attr
if len(redundant_rows)>0:
  decision_attr=np.delete(decision_attr,redundant_rows,axis=0)


Done......


In [20]:
print(decision_attr)

[0 2 1 2 1 1 2 1]


In [21]:
print(new_train_attributes)

[[0 2]
 [1 1]
 [0 1]
 [1 2]
 [0 0]
 [2 1]
 [1 1]
 [1 0]]
